In [71]:
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Rescaling
from keras.layers import BatchNormalization

### Variables

In [72]:
# No tocar
num_clases = 26
# La foto es 128x96
xpixel = 96
ypixel = 128
# Tocar
n_neuronas_conv1 = 64
n_neuronas_conv2 = 128
n_neuronas_conv3 = 256
l_rate = 0.001  # empezar en 0.001 e ir bajando para el estudio
epoch = 15
batch = 1  # Realmente en 1 esta bien esto es mas para tiempos de ejecucion con grandes cantidades de datos

### Meter Datos

In [73]:
def meterdatos():
    directorio = r'../Dataset/'
    df = pd.read_csv(directorio + 'driver_imgs_list.csv')

    fotos = df['img'].values
    conductores = df['subject'].values
    clases = df['classname'].values
    # lo de rutas es magico la vd jejejejej
    rutas = clases + '/' + fotos
    dataset = pd.DataFrame([rutas , conductores])

    imagenes = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range = [0.6, 1.4],
    brightness_range = [0.3, 1.4]
        ) #Aqui podemos hacer augmentation

    
    datasetEntero = dataset.T
    datasetEntero.columns = ['fotos','conductores'] #que sino no  sabe que cojer xcol e ycol
 
    directorio2 = r'../Dataset/imgs/train/'

    entrenamiento = imagenes.flow_from_dataframe(
        dataframe=datasetEntero,
        directory=directorio2,
        x_col="fotos",
        y_col="conductores",
        subset="training",
        batch_size=batch,
        seed=123,
        shuffle=True,
        class_mode="categorical",
        target_size=(xpixel,ypixel)
    )
    
    validacion = imagenes.flow_from_dataframe(
        dataframe=datasetEntero,
        directory=directorio2,
        x_col="fotos",
        y_col="conductores",
        subset="validation",
        batch_size=batch,
        seed=123,
        shuffle=True,
        class_mode="categorical",
        target_size=(xpixel,ypixel)
    )
    return entrenamiento , validacion


In [74]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range = [0.6, 1.4],
    brightness_range = [0.3, 1.4]
        )

### Modelar Red

In [75]:
def Modelar_red_trampas(x, y, lr): # red original con los valores de MLP cambiados
    model = Sequential()
    # Capa input
    # Como no he rescalado nada usamos esta capa para normalizar los datos
    model.add(Rescaling(1./255, input_shape=(x, y, 3)))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))  # drop 50% of neurons
    # capa de salida(softmaxx)
    model.add(Dense(num_clases, activation='softmax'))

    # Compilamos
    # https://keras.io/api/optimizers/adam/ ?
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="categorical_crossentropy",
                  optimizer=adam, metrics=['accuracy'])

    return model

In [76]:
# sin pooling
def modelar_red_2(x, y, lr): # red original con los valores de MLP cambiados
    model = Sequential()
    # Capa input
    # Como no he rescalado nada usamos esta capa para normalizar los datos
    model.add(Rescaling(1./255, input_shape=(x, y, 3)))
    
    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(x, y, 3)))
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))  # drop 50% of neurons
    # capa de salida(softmaxx)
    model.add(Dense(num_clases, activation='softmax'))

    # Compilamos
    # https://keras.io/api/optimizers/adam/ ?
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="categorical_crossentropy",
                  optimizer=adam, metrics=['accuracy'])

    return model

In [77]:
# 1 pooling
def modelar_red_3(x, y, lr): # red original con los valores de MLP cambiados
    model = Sequential()
    # Capa input
    # Como no he rescalado nada usamos esta capa para normalizar los datos
    model.add(Rescaling(1./255, input_shape=(x, y, 3)))
    
    model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(x, y, 3)))
    model.add(MaxPooling2D(pool_size=(4, 4)))
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))  # drop 50% of neurons
    # capa de salida(softmaxx)
    model.add(Dense(num_clases, activation='softmax'))

    # Compilamos
    # https://keras.io/api/optimizers/adam/ ?
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="categorical_crossentropy",
                  optimizer=adam, metrics=['accuracy'])

    return model

In [78]:
# obtencion de datos modo:trampa
import cv2 
import os
from sklearn.preprocessing import LabelBinarizer
from collections import OrderedDict
from sklearn.model_selection import train_test_split
import numpy as np

images = []
nombres = []
contador = 0

for i in range (10):
    contador = 0
    print(i)
    for filename in os.listdir('../Dataset/imgs/train/c'+str(i)+'/'):
        if contador < 2000:
            img = cv2.imread(os.path.join('../Dataset/imgs/train/c'+str(i)+'/',filename))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img,(128,96))
            images.append(img/255)
            nombres.append(filename)
        contador += 1

#---------------

df_1= pd.read_csv("../Dataset/driver_imgs_list.csv")
df_2= pd.DataFrame()
df_2['array'] = images
df_2['img'] = nombres

df_final = df_1.merge(df_2, on="img", how="left")

df_final = df_final.dropna()

#---------------

imagenes = df_final['array'].tolist()
personas = LabelBinarizer().fit_transform(df_final.subject)

array_personas = df_final['subject'].to_numpy()
array_personas = OrderedDict.fromkeys(array_personas)
array_personas = list(filter(None, array_personas))

#---------------

entrenamiento_entradas,validacion_entradas,entrenamiento_salidas,validacion_salidas = train_test_split(imagenes, personas,test_size=0.2)

entrenamiento_entradas = np.array(entrenamiento_entradas).astype(float)
validacion_entradas = np.array(validacion_entradas).astype(float)

0
1
2
3
4
5
6
7
8
9


In [79]:
# 1 pooling
def modelar_red_4(x, y, lr): # red original con los valores de MLP cambiados
    model = Sequential()

  # Capas convolucionales
    model.add(Conv2D(32, kernel_size=(3, 3),strides=1,activation='relu',padding='same',input_shape=(x,y,3)))
    model.add(MaxPooling2D((2,2),padding='same'))
    model.add(Conv2D(64, kernel_size=(3, 3),strides=1,activation='relu',padding='same'))
    model.add(MaxPooling2D((2,2),padding='same'))
    model.add(Conv2D(128, kernel_size=(3, 3),strides=1,activation='relu',padding='same'))
    model.add(MaxPooling2D((2,2),padding='same'))
    model.add(Dropout(0.5))
    model.add(Flatten())

    # Capas del MLP
    model.add(Dense(64, activation='relu'))
    model.add(Dense(26, activation='softmax'))

    # Compilamos
    # https://keras.io/api/optimizers/adam/ ?
    adam = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="categorical_crossentropy",
                  optimizer=adam, metrics=['accuracy'])

    return model

### Entrenar

In [80]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch, logs={}):
        if(logs.get('loss') <= 0.2):
            self.model.stop_training = True

def Entrenar(m,e,v ,epo, b):
    # Para parar segun el criterio de la pr
    elcallback = myCallback()
    
    history = m.fit(
        x = e,
        validation_data = v,
        epochs=epo,
        batch_size=b,
        verbose=1,  # Esto te imprime un progress bar con informacion
        shuffle=True,
        callbacks= elcallback
    )
    return history

### Evaluacion

In [81]:
def Evaluar(m, e_test):
    return m.evaluate(e_test,verbose=1)

### Funciones Esteticas

In [82]:
# Pintamos nuestro modelo

def pintarmodelo(modelo):
    tf.keras.utils.plot_model(
        modelo,
        to_file='model.png',
        show_shapes=False,
        show_dtype=False,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=False,
        dpi=96,
        layer_range=None,
        show_layer_activations=False
    )

# Pintamos imagenes
def mostrarimagenes(imagenes, batch):
    img, labels = next(imagenes)
    for i in range(batch):
        plt.imshow(img[i, :, :, :])
        plt.title(labels[i])
        plt.show()


### Guardar y cargar el modelo neuronal deseado

In [83]:
def cargarmodelo():
    model = ker.models.load_model('path/to/location')  # Carga modelos
    return model
    
def guardarmodelo():
    model.save('ModeloGuardado')  # guarda el modelo en la ruta que desees

### Preparación para entrenar

In [84]:
train, validation = meterdatos()
# Puedes cambiar la funcion para elegir otro modelo
# model = Modelar_red_trampas(xpixel, ypixel, l_rate)

Found 17940 validated image filenames belonging to 26 classes.
Found 4484 validated image filenames belonging to 26 classes.


### Ejecución

In [85]:
# si no satisface al callback esto parara
''' 
history = Entrenar(model, train, validation, epoch, batch)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
'''


" \nhistory = Entrenar(model, train, validation, epoch, batch)\n\nplt.plot(history.history['accuracy'], label='accuracy')\nplt.plot(history.history['val_accuracy'], label = 'val_accuracy')\nplt.plot(history.history['loss'], label='loss')\nplt.plot(history.history['val_loss'], label = 'val_loss')\nplt.xlabel('Epoch')\nplt.ylabel('Accuracy')\nplt.legend(loc='lower right')\n"

### Evaluación

In [86]:
'''
train_loss, train_acc = Evaluar(model,train)
val_loss, val_acc = Evaluar(model,validation)
'''

'\ntrain_loss, train_acc = Evaluar(model,train)\nval_loss, val_acc = Evaluar(model,validation)\n'

## Pruebas de diferentes modelos de red

In [87]:
'''
model = modelar_red_2(xpixel, ypixel, l_rate)
history = Entrenar(model, train, validation, epoch, batch)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
'''

"\nmodel = modelar_red_2(xpixel, ypixel, l_rate)\nhistory = Entrenar(model, train, validation, epoch, batch)\n\nplt.plot(history.history['accuracy'], label='accuracy')\nplt.plot(history.history['val_accuracy'], label = 'val_accuracy')\nplt.plot(history.history['loss'], label='loss')\nplt.plot(history.history['val_loss'], label = 'val_loss')\nplt.xlabel('Epoch')\nplt.ylabel('Accuracy')\nplt.legend(loc='lower right')\n"

In [88]:
'''
model = modelar_red_3(xpixel, ypixel, l_rate)
history = Entrenar(model, train, validation, epoch, batch)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
'''

"\nmodel = modelar_red_3(xpixel, ypixel, l_rate)\nhistory = Entrenar(model, train, validation, epoch, batch)\n\nplt.plot(history.history['accuracy'], label='accuracy')\nplt.plot(history.history['val_accuracy'], label = 'val_accuracy')\nplt.plot(history.history['loss'], label='loss')\nplt.plot(history.history['val_loss'], label = 'val_loss')\nplt.xlabel('Epoch')\nplt.ylabel('Accuracy')\nplt.legend(loc='lower right')\n"

In [90]:
'''
model = modelar_red_4(xpixel, ypixel, l_rate)
history = Entrenar(model,train,validation, epoch, batch)
model = modelar_red_4(xpixel,ypixel,l_rate)
# Nueva red
'''
model = modelar_red_4(xpixel,ypixel,l_rate)
history = model.fit(entrenamiento_entradas, 
entrenamiento_salidas,epochs=15,verbose=1,
validation_data=(validacion_entradas, validacion_salidas), callbacks=myCallback())

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

Epoch 1/15
 2874/17940 [===>..........................] - ETA: 2:10 - loss: 1.1437 - accuracy: 0.6552

KeyboardInterrupt: 